In [1]:
import json
from pprint import pprint

import jupyter_black
from dotenv import load_dotenv

from baml_agents.utils import BamlGenerator
from notebooks._utils import filter_tools_by_name, run_cli_command

jupyter_black.load()  # Python linter
load_dotenv()

True

# Automatically generate BAML class definitions from MCP servers

### Prerequisites

We'll use the `mcpt` command-line interface to interact with MCP servers. It functions much like `curl`, allowing us to query the servers easily.

- Install it from here: [github.com/f/mcptools](https://github.com/f/mcptools)

### Showing around the MCP Server

Let's see all available tools on the `uvx mcp-server-calculator` [MCP server](https://github.com/githejie/mcp-server-calculator). Server will be downloaded as a python package by `uvx` and run locally until we get the data we need.

In [2]:
# Show all the tools in the server
!mcpt tools uvx mcp-server-calculator

calculate(expression:str)
     Calculates/evaluates the given expression.



Run the `calculate` tool:

In [3]:
# Use the calculator tool
!mcpt call calculate -p '{"expression": "123.4 * 2"}' uvx mcp-server-calculator

246.8


### Generating BAML code from MCP Tools Schema

Let's get the `calculate` MCP Tool schema from the `uvx mcp-server-calculator` MCP Server

In [4]:
mcp_server = "uvx mcp-server-calculator"
mcp_tools_schema = run_cli_command(f"mcpt tools --format json {mcp_server}")
parsed_schema = json.loads(mcp_tools_schema)
pprint(parsed_schema)

{'tools': [{'description': 'Calculates/evaluates the given expression.',
            'inputSchema': {'properties': {'expression': {'title': 'Expression',
                                                          'type': 'string'}},
                            'required': ['expression'],
                            'title': 'calculateArguments',
                            'type': 'object'},
            'name': 'calculate'}]}


Then, generate BAML:

In [5]:
print(BamlGenerator().from_mcp_tools_schema(parsed_schema))

class Calculatearguments {
  expression string
}

class CalculateTool {
  name "calculate" @description(#"Calculates/evaluates the given expression."#)
  arguments CalculateArguments
}


Note: You can fully customize the produced BAML code:

In [6]:
gen = BamlGenerator(
    tool_name_field="mcp_tool_id",
    tool_args_field="params",
    tool_name=lambda name, **_: name,
    args_class=lambda name, **_: f"ParamsAAA{name.capitalize()}Action",
    tool_class=lambda name, **_: f"{name.capitalize()}Action",
    description=lambda **_: "",  # Making it empty will remove it
    alias=lambda name, schema, place, **_: (
        schema.get("title", name) if place == "property" else "name"
    ),
    prop_name=lambda name, **_: name,
)
print(gen.from_mcp_tools_schema(parsed_schema))

class Paramsaaacalculateaction {
  expression string @alias("Expression")
}

class CalculateAction {
  mcp_tool_id "calculate" @alias("name")
  params ParamsAAACalculateAction
}


Let's invoke the tool:

In [7]:
from baml_client import b
from baml_client.types import CalculateAction

goal = "add first three prime numbers"
action: CalculateAction = b.GetCalculateAction(goal)
action

CalculateAction(mcp_tool_id='calculate', params=ParamsCalculateAction(expression='2 + 3 + 5'))

In [8]:
params = action.params.model_dump_json()
command = f"mcpt call {action.mcp_tool_id} -p '{params}' {mcp_server}"
print(f"Running cli command:\n> {command}")
run_cli_command(command)

Running cli command:
> mcpt call calculate -p '{"expression":"2 + 3 + 5"}' uvx mcp-server-calculator


'10'

### Generating BAML code from MCP Tools Schema

Step 1: Choose an MCP Server to use (provide its URL or the command to run it)

In [9]:
mcp_server = "uvx arxiv-mcp-server"

Step 2: BAML types are automatically generated for all the tools in the server:

In [10]:
tools_schema = run_cli_command(f"mcpt tools --format json {mcp_server}")
tools_schema = filter_tools_by_name(tools_schema, "search_papers")
tools_schema

{'tools': [{'description': 'Search for papers on arXiv with advanced filtering',
   'inputSchema': {'properties': {'categories': {'items': {'type': 'string'},
      'type': 'array'},
     'date_from': {'type': 'string'},
     'date_to': {'type': 'string'},
     'max_results': {'type': 'integer'},
     'query': {'type': 'string'}},
    'required': ['query'],
    'type': 'object'},
   'name': 'search_papers'}]}

In [11]:
print(BamlGenerator().from_mcp_tools_schema(tools_schema))

class Searchpapersarguments {
  query string
  categories string?[]
  date_from string?
  date_to string?
  max_results int?
}

class SearchPapersTool {
  name "search_papers" @description(#"Search for papers on arXiv with advanced filtering"#)
  arguments SearchPapersArguments
}


Step 3: Copy the generated BAML code to baml_src, create a new BAML function and run it

In [12]:
from baml_client import b

goal = "want to build a site but never coded before"
tool = b.GetSearchStoriesTool(goal)
tool

SearchStoriesTool(name='search_stories', arguments=SearchStoriesArguments(query='how to build a website no coding experience', num_results=10, search_by_date=False))

In [13]:
command = f"mcpt call {tool.name} -p '{tool.arguments.model_dump_json()}' {mcp_server}"
print(command)

mcpt call search_stories -p '{"query":"how to build a website no coding experience","num_results":10,"search_by_date":false}' uvx arxiv-mcp-server


In [14]:
result = run_cli_command(command)
print(f"Running cli command:\n> {command}\n{result}")

Running cli command:
> mcpt call search_stories -p '{"query":"how to build a website no coding experience","num_results":10,"search_by_date":false}' uvx arxiv-mcp-server
Error: Unknown tool search_stories
